# Datos del registro  unico de victimas

Estos son datos del Registro unico de victimas.  http://rni.unidadvictimas.gov.co/RUV. Es dificil saber con total seguridad que son estos archivos porque no esta bien definido que hay en ellos.

In [1]:
import pandas as pd
import pickle
import time
# Pandas cuantas filas muestra?
pd.set_option('display.max_rows', 100)

In [2]:
ls ./data/

Reporte Declaracion.xlsx
Reporte Desplazados por Enfoque Anio Municipio.xlsx
Reporte Hogares Desplazados.xlsx
Reporte Ocurrencia Hecho Victimizante.xlsx
Reporte Personas Desplazadas.xlsx
Reporte Personas por Municipio Declaracion.xlsx
Reporte Personas por Municipio de Ocurrencia.xlsx
Reporte Personas por Municipio de Siniestro - Directos e Indirectos..xlsx
Reporteador.xlsx


# Unificar los datos
Voy a usar el archivo conocido como reportador por el mas facil para un analysis simple. Tambien se puede hacer con los otros pero requirer un poquito mas de trabajo. Puede ser interesante si se quiere hacer una analysis anualizado

In [3]:
# Reporte_de_ocurrencia = pd.read_excel('./data/Reporte Ocurrencia Hecho Victimizante.xlsx')
# Reporte_Declaracion = pd.read_excel('./data/Reporte Declaracion.xlsx')
# Reporte_por_deplazados_por_municipio = pd.read_excel('./data/Reporte Desplazados por Enfoque Anio Municipio.xlsx')
Reporteador = pd.read_excel('./data/Reporteador.xlsx')
Reporteador = Reporteador.rename(columns={'MUNICIPIO DECLARACION':'municipio', 'DEPARTAMENTO DECLARACION':'departamento'})
del Reporteador['DANE DECLARACION']
del Reporteador['TIPO DE VICTIMA']

## remove data that is duplicated

In [4]:
Reporteador = Reporteador.groupby(['departamento', 'municipio']).sum().reset_index()

# Verificando la consistencia de los datos
Parece que el Reporte de archivo Reporte Declaracion y el Reporteador son los mas amplios. Uno se asombra que en Colombia la mayoria de las victimas son desplazados.

In [5]:
# print(Reporte_de_ocurrencia['TOTAL'].sum())
# print(Reporte_Declaracion['TOTAL'].sum())
print(Reporteador['TOTAL'].sum())
# print(Reporte_por_deplazados_por_municipio['EXPULSION'].sum())

8845389


# Unificando  con los datos de votacion

In [14]:
Votos_municipio = pickle.load(open( './plebicito_municipios_20161010.pickle', 'rb'))
Votos_municipio = Votos_municipio.rename(columns={'nombre':'municipio'})

In [15]:
print('Votos Si {:d}'.format(Votos_municipio['Si'].sum()))
print('Votos No {:d}'.format(Votos_municipio['No'].sum()))

Votos Si 5561052
Votos No 5667641


In [16]:
Votos_municipio_y_victimas = Votos_municipio.merge(Reporteador, on=['municipio', 'departamento'])

In [17]:
# Me gusta ver los datos por departamento
Votos_municipio_y_victimas = Votos_municipio_y_victimas.sort_values(by='departamento')
Votos_municipio_y_victimas = Votos_municipio_y_victimas.reset_index()

In [18]:
print(Votos_municipio_y_victimas.loc[0:10,['municipio', 'departamento', 'Si', 'No', 'TOTAL']])

     municipio departamento      Si      No   TOTAL
0      LETICIA     AMAZONAS    4403    4463    2467
1     TARAPACA     AMAZONAS     265      68      16
2      MURINDO    ANTIOQUIA    1073      44    5089
3   MONTEBELLO    ANTIOQUIA     953     776    2258
4     MEDELLIN    ANTIOQUIA  253548  431173  625530
5    MARINILLA    ANTIOQUIA    5481   11993   10560
6        MACEO    ANTIOQUIA     777    1065    2523
7     LIBORINA    ANTIOQUIA     933    1674    4535
8       JERICO    ANTIOQUIA     957    2341     469
9    CHIGORODO    ANTIOQUIA    6322    5745   45152
10      ITAGUI    ANTIOQUIA   35166   59627   17162


# Preservar

In [19]:
import time
day = time.strftime("%Y%m%d")

In [22]:
Votos_municipio_y_victimas.to_csv('plebicito_municipios_y_victimas{}.csv'.format(day))
Votos_municipio_y_victimas.to_pickle('plebicito_municipios_y_victimas{}.pickle'.format(day))